# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Run the base watershed model with PETSc on one processor

In [ ]:
import pathlib as pl
import sys

import flopy
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append("../../base/watershed/")
from defaults import (
    figsize,
    geometry,
    get_base_dir,
    get_simulation_cell_count,
    string2geom,
)

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

## Parallel settings

Set `path_to_mf6` to the path of parallel MODFLOW 6 (`path\to\mf6`) if it is not in your `PATH` otherwise set to `None`.

In [ ]:
path_to_mf6 = None

## Load the base watershed model

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

### Count the number of active cells

In [ ]:
ncells, nactive = get_simulation_cell_count(sim)
print("nr. of cells:", ncells, ", active:", nactive)

### Change the model directory and write the base watershed model files

In [ ]:
model_dir = get_base_dir()
print(f"{model_dir}")

In [ ]:
sim.set_sim_path(model_dir)

In [ ]:
sim.write_simulation()

## Run the base watershed model

In [ ]:
sim.run_simulation(processors=1, silent=False)

### Plot the results

In [ ]:
boundary_polygon = string2geom(geometry["boundary"])
bp = np.array(boundary_polygon)

stream_segs = (
    geometry["streamseg1"],
    geometry["streamseg2"],
    geometry["streamseg3"],
    geometry["streamseg4"],
)
sgs = [string2geom(sg) for sg in stream_segs]

In [ ]:
gwf = sim.get_model()
times = gwf.output.head().get_times()
base_head = np.squeeze(gwf.output.head().get_data(totim=times[-1]))

In [ ]:
fig = plt.figure(figsize=figsize)
pmv = flopy.plot.PlotMapView(model=gwf, layer=0)
pmv.plot_array(base_head)

for sg in sgs:
    sa = np.array(sg)
    pmv.ax.plot(sa[:, 0], sa[:, 1], "b-")